# Section: Securing Federated Learning

- Lesson 1: Trusted Aggregator
- Lesson 2: Intro to Additive Secret Sharing
- Lesson 3: Intro to Fixed Precision Encoding
- Lesson 4: Secret Sharing + Fixed Precision in PySyft
- Final Project: Federated Learning wtih Encrypted Gradient Aggregation

# Lesson: Federated Learning with a Trusted Aggregator

In the last section, we learned how to train a model on a distributed dataset using Federated Learning. In particular, the last project aggregated gradients directly from one data owner to another. 

However, while in some cases it could be ideal to do this, what would be even better is to be able to choose a neutral third party to perform the aggregation.

As it turns out, we can use the same tools we used previously to accomplish this.

# Project: Federated Learning with a Trusted Aggregator

### Step 1: Create Data Owners

In [4]:
!pip install syft
!pip install torchvision

     |████████████████████████████████| 225kB 3.5MB/s 
     |████████████████████████████████| 204kB 64.3MB/s 
     |████████████████████████████████| 81kB 34.5MB/s 
     |████████████████████████████████| 1.4MB 57.9MB/s 
     |████████████████████████████████| 450kB 60.9MB/s 
     |████████████████████████████████| 389kB 45.8MB/s 
     |████████████████████████████████| 256kB 51.2MB/s 
     |████████████████████████████████| 276kB 39.3MB/s 
     |████████████████████████████████| 51kB 25.9MB/s 
     |████████████████████████████████| 122kB 51.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/ad/9a/f4/3105b5209674ac77fcca7fede95184c62a95df0196888e0e76
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built zstd pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Found existing installation: msgpack 0.5.6
    Uninstalling msgpack-0.5.6:
     

In [23]:
import syft as sy

import torch
from torchvision import datasets, transforms
import helper

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
# Download and load the training data
trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Download and load the test data
testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)
import torch as th
from torch import nn, optim

hook = sy.TorchHook(th)

RuntimeError: ignored

In [6]:
# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

# this step is important in the real-world application.
# You need to inform the workers of others existance
# you will probably have an ssh or http worker, not a virtual worker.

bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

W0711 22:25:49.710528 139805584787328 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0711 22:25:49.717165 139805584787328 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0711 22:25:49.720453 139805584787328 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0711 22:25:49.722541 139805584787328 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0711 22:25:49.724094 139805584787328 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0711 22:25:49.725016 139805584787328 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secure_worker #objects:0>

In [0]:
# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)


### Step 2: Create the Model

In [0]:
# Iniitalize A Toy Model
model = nn.Linear(2,1)

### Step 3: Send a Copy of The Model to Alice and Bob

In [0]:
bobs_model = model.copy().send(bob) 
alices_model = model.copy().send(alice) 

bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)


### Step 4: Train Bob's and Alice's Models (in parallel)

In [0]:
for i in range(10):

    # Train Bob's Model
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target)**2).mean()
    bobs_loss.backward()

    bobs_opt.step()
    bobs_loss = bobs_loss.get().data
    print(f'Bob loss {bobs_loss}')

    # Train Alice's Model
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target)**2).mean()
    alices_loss.backward()

    alices_opt.step()
    alices_loss = alices_loss.get().data
    print(f'alice loss {alices_loss}')


Bob loss 0.18725763261318207
alice loss 0.8352035284042358
Bob loss 0.13208825886249542
alice loss 0.2517654597759247
Bob loss 0.09977314621210098
alice loss 0.07879725098609924
Bob loss 0.08008083701133728
alice loss 0.02726730890572071
Bob loss 0.06742317974567413
alice loss 0.011686650104820728
Bob loss 0.05874332785606384
alice loss 0.006767766084522009
Bob loss 0.052364666014909744
alice loss 0.005028928630053997
Bob loss 0.04736354947090149
alice loss 0.004255000501871109
Bob loss 0.043227389454841614
alice loss 0.003788722213357687
Bob loss 0.03966829553246498
alice loss 0.0034338063560426235


### Step 5: Send Both Updated Models to a Secure Worker

In [0]:
alices_model.move(secure_worker) # the move function iterates on all objects in alice and call send on it. NOTEICE: inline function
bobs_model.move(secure_worker)


In [0]:
secure_worker._objects # can you identify those objects in the Secure worker?

{5592503954: Parameter containing:
 tensor([[ 0.1004, -0.1115]], requires_grad=True),
 12931965454: Parameter containing:
 tensor([0.9601], requires_grad=True),
 23456639407: Parameter containing:
 tensor([0.2409], requires_grad=True),
 70371717135: Parameter containing:
 tensor([[-0.2769, -0.3634]], requires_grad=True)}

### Step 6: Average The Models

In [0]:
# A pytorch trick: to set the weights: you have to either access the data[idx] 
# or wrap the set function with (with torch.no_grad():)
# below we illustrate the first way
# the next cell demonstrates the complete way:

model.weight.data[0].set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
model.bias.data[0].set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())

tensor([0.6005])

### Step 7: The complete solution over multiple training sessions

In [0]:
iterations = 10
worker_iters = 5

for a_iter in range(iterations):

    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)

    for wi in range(worker_iters):
        # Train Bob's Model
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target) ** 2).sum()
        bobs_loss.backward()

        bobs_opt.step()
        bobs_loss = bobs_loss.get().data

        # Train Alice's Model
        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target) ** 2).sum()
        alices_loss.backward()

        alices_opt.step()
        alices_loss = alices_loss.get().data

    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)

    with th.no_grad():

        model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
        model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(0.0843) Alice:tensor(0.0074)
Bob:tensor(0.0548) Alice:tensor(0.0018)
Bob:tensor(0.0368) Alice:tensor(0.0005)
Bob:tensor(0.0255) Alice:tensor(8.4883e-05)
Bob:tensor(0.0180) Alice:tensor(3.6544e-06)
Bob:tensor(0.0130) Alice:tensor(5.6073e-06)
Bob:tensor(0.0095) Alice:tensor(2.2419e-05)
Bob:tensor(0.0070) Alice:tensor(3.4822e-05)
Bob:tensor(0.0053) Alice:tensor(4.0013e-05)
Bob:tensor(0.0040) Alice:tensor(3.9748e-05)


In [0]:
preds = model(data)
loss = ((preds - target) ** 2).mean()

In [0]:
print(preds)
print(target)
print(loss.data)

tensor([[0.1718],
        [0.1375],
        [0.8275],
        [0.7932]], grad_fn=<AddmmBackward>)
tensor([[0.],
        [0.],
        [1.],
        [1.]], requires_grad=True)
tensor(0.0302)


# Lesson: Intro to Additive Secret Sharing

While being able to have a trusted third party to perform the aggregation is certainly nice, in an ideal setting we wouldn't have to trust anyone at all. This is where Cryptography can provide an interesting alterantive. 

Specifically, we're going to be looking at a simple protocol for Secure Multi-Party Computation called Additive Secret Sharing. This protocol will allow multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x

In [0]:
x = 5

Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [0]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

5

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted). 

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [0]:
bob_x_share *= 2
alice_x_share *= 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

10

This even works for addition between two shared values!!

In [0]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

12

As you can see, we just added two numbers together while they were still encrypted ( you are the dealer, thus still can see --and know everything-- but Alice and Bob did not really know anything about the overall deal --the sum of the numbers; the secrete--)

In this case, however, to keep the secrete is not about the relaiability / integrity of the person who holds it, rather the fact that those shareholders do not know each other!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. Decryption happens by summing all the shares together MODULUS some constant.

In [0]:
x = 5

Q = 9923 # the little fix; a very large prime number

bob_x_share = 1525 # give bob a random number.. try with negatives

alice_x_share = Q - bob_x_share + x

print(f'Bob share= {bob_x_share}')
print(f'Alice share= {alice_x_share}')
print(f'Add both shares= {bob_x_share + alice_x_share}... Wrong secrete')

Bob share= 1525
Alice share= 8403
Add both shares= 9928... Wrong secrete


In [0]:
# Decryption happens by summing all the shares together MODULUS some constant

(bob_x_share + alice_x_share) % Q

5

So now, as you can see, both shares are wildly larger than the number being shared, meaning that individual shares no longer leak this inforation. However, all the properties we discussed earlier still hold! (addition, encryption, decryption, etc.)

# Project: Build Methods for Encrypt, Decrypt, and Add 

In this project, you must take the lessons we learned in the last section and write general methods for encrypt, decrypt, and add. Store shares for a variable in a tuple like so:  `x_share = (2,5,7)`

Even though normally those shares would be distributed among several workers, you can store them in ordered tuples like this for now.

In [0]:
import random

Q = 23740629843760239486723

# accepts a secrete and shares it among n_shahre, returns a tuple with n_share shares
def encrypt(x, n_share=3):
    
    shares = list()
    
    for i in range(n_share-1):
        shares.append(random.randint(0,Q))
        
    shares.append(Q - (sum(shares) % Q) + x)
    
    return tuple(shares)

  # accepts a tuple of shares and return the decrypted values
def decrypt(shares):
    return sum(shares) % Q

In [0]:
shares = encrypt(x=7, n_share=10)
shares

(15899097772522877364249,
 11367675966508840117530,
 4900233884997846532104,
 16630337867297688321351,
 6676581143161867586917,
 965082419012027105831,
 10007139259242749719591,
 2826785571199337361780,
 9849092175711078026532,
 15840493315386645811014)

In [0]:
decrypt(shares)

7

In [0]:
def add(a, b):
    c = list()
    assert(len(a) == len(b))
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
    return tuple(c)

In [0]:
# run this block of code multiple times, what do you notice baout x and y?

x = encrypt(5)
print(x)
y = encrypt(7)
print(y)

z = add(x,y)
decrypt(z)

(19450006935608221574999, 18583217581987782711521, 9448035169924474686931)
(18876374866682609093470, 10862363173324101889542, 17742521647513767990441)


12

# Lesson: Intro to Fixed Precision Encoding

As you may remember, our goal is to aggregate gradients using this new Secret Sharing technique. However, the protocol we've just explored in the last section uses positive integers. However, our neural network weights are NOT integers. Instead, our weights are decimals (floating point numbers).

Not a huge deal! We just need to use a fixed precision encoding, which lets us do computation over decimal numbers using integers!

In [0]:
BASE= 10  # integer numbers (normal numbers)
PRECISION= 3  # how many porecisions after the decimal point to encode
Q = 23740629843760239486723 

In [0]:
def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION

In [0]:
encode(-0.600)

9323

In [0]:
decode(9323)

-0.6

In [0]:
# run this cell multiple times and notice the changes

x = encrypt(encode(5.5))
y = encrypt(encode(5.5))
z = add(x,y)


print(f'Value of x = {x}')
print(f'Value of y = {y}')
print(f'Value of sum = {z}')
print(f'decoded sum = {decode(decrypt(z))}')

Value of x = (3415, 7020, 14911)
Value of y = (4595, 7707, 13044)
Value of sum = (8010, 4804, 8109)
decoded sum = 1.077


# Lesson: Secret Sharing + Fixed Precision in PySyft

While writing things from scratch is certainly educational, PySyft makes a great deal of this much easier for us through its abstractions.

In [3]:
# Its a good idea to restrat the environment at this point

import syft as sy
import torch as th
from torch import nn, optim

hook = sy.TorchHook(th)

# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")


bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

ModuleNotFoundError: ignored

In [0]:
x = th.tensor([1,2,3,4,5]) ##### the following steps work for integer values only

### Secret Sharing Using PySyft

We can share using the simple` .share()` method!

In [0]:
x = x.share(bob, alice, secure_worker) # implemets additive secrete shares
x # it's a pointer to three secrete shares

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:86660417298 -> bob:21697835486]
	-> (Wrapper)>[PointerTensor | me:22601586543 -> alice:37627061481]
	-> (Wrapper)>[PointerTensor | me:52553544203 -> secure_worker:33776213713]
	*crypto provider: me*

In [0]:
bob._objects # take a look at one of the shares.. Large random numbers --encrypted

{21697835486: tensor([ 931215927921640947,  459606633707528827, 2077348708747142219,
         2518050637076306642, 2250413087966072035])}

and as you can see, Bob now has one of the shares of x! Furthermore, we can still call addition in this state, and PySyft will automatically perform the remote execution for us!

In [0]:
y = x + x
y

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:15477851456 -> bob:77734773635]
	-> (Wrapper)>[PointerTensor | me:54003512272 -> alice:27089881245]
	-> (Wrapper)>[PointerTensor | me:22296853755 -> secure_worker:81023290460]
	*crypto provider: me*

In [0]:
bob._objects # bob has another tensor.. why?

{21697835486: tensor([ 931215927921640947,  459606633707528827, 2077348708747142219,
         2518050637076306642, 2250413087966072035]),
 77734773635: tensor([1862431855843281894,  919213267415057654, 4154697417494284438,
         5036101274152613284, 4500826175932144070])}

In [0]:
y.get() # notice that we have doubled the values on each worker!

tensor([ 2,  4,  6,  8, 10])


### Fixed Precision using PySyft

We can also convert a tensor to fixed precision using `.fix_precision()`
This is important when doing FL since the values you are sharing are decimal numbers.

**Q: what values are we sharing? what are the shared secretes?**

In [0]:
x = th.tensor([0.1, 0.2, 0.3, 0.4]) # a decimal tensor!

In [0]:
x

tensor([0.1000, 0.2000, 0.3000, 0.4000])

In [0]:
x = x.fix_prec() # call this function to encode the decimal tensor using fixed precision
x # notice that this is a tensor chain <-------

(Wrapper)>FixedPrecisionTensor>tensor([100, 200, 300, 400])

In [0]:
print(f'Type of x {type(x)}')
print(f'Type of x.child {type(x.child)}')
print(f'Type of x.child.child {type(x.child.child)}') # <-- this is how you get the data from a fixed precision tensor

Type of x <class 'syft.frameworks.torch.tensors.interpreters.native.Tensor'>
Type of x.child <class 'syft.frameworks.torch.tensors.interpreters.precision.FixedPrecisionTensor'>
Type of x.child.child <class 'syft.frameworks.torch.tensors.interpreters.native.Tensor'>


In [0]:
# to decode the previous tensor
print(x.float_prec())

tensor([0.1000, 0.2000, 0.3000, 0.4000])


In [0]:
y = x + x
y

(Wrapper)>FixedPrecisionTensor>tensor([200, 400, 600, 800])

In [0]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000, 0.8000])

### Shared Fixed Precision

And of course, we can combine the two!

In [0]:
x = th.tensor([0.1, 0.2, 0.3])

In [0]:
x = x.fix_prec().share(bob, alice)
x

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:43948102131 -> bob:41549875499]
	-> (Wrapper)>[PointerTensor | me:98889542648 -> alice:96286304541]
	*crypto provider: me*

In [0]:
bob._objects

{1896941596: tensor([3337927225947607575, 4115017391059871510, 5758453651456544530]),
 19122572715: tensor([1981985574526026572, 2759075739638290507, 4402512000034963527]),
 21697835486: tensor([ 931215927921640947,  459606633707528827, 2077348708747142219,
         2518050637076306642, 2250413087966072035]),
 41549875499: tensor([3701295438721311877,  945318970846132605,   20024228591329413])}

In [0]:
y = x + x

In [0]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])

# Final Project: Federated Learning with Encrypted Gradient Aggregation

Reuse your project from the Secuered Agggregator above to train the same model using the FL appraoch, but this time use the Additive Sharing Encryption so that the participating members have access to their own models only. There is no secure aggregator in this senario. 
Include four members in this project (Alice, Bob, Ted, and Carol). 

Hint: we will *share* the model with the shareholders, do addition on the encrypted models, and then aggregate the final model locally. 

In [18]:
# create cells as needed. Separate your solution into logical

bob.clear_objects()
alice.clear_objects()
ted.clear_objects()
carol.clear_objects()
# cryptoprovider.clear_objects()

hook = sy.TorchHook(th)  
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
ted = sy.VirtualWorker(hook, id="ted")
carol = sy.VirtualWorker(hook, id="carol")
cryptoprovider = sy.VirtualWorker(hook, id="cryptoprovider")

# Make each worker aware of the other workers
bob.add_workers([alice,ted, carol])
alice.add_workers([bob,ted, carol])
carol.add_workers([bob, alice, ted])
ted.add_workers([bob, alice, carol])

# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0],[1.,1],[0,1],[1,0],[0,0],[1.,1],[0,1],[1,0],[0,0], [1.,1],[0,1],[1,0],[0,0] ], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0], [1.],[1], [0], [0],[1.],[1], [0], [0], [1.],[1], [0], [0]], requires_grad=True)


workers = [bob, alice, carol, ted]
model = th.nn.Linear(2,1)
model = model.fix_precision().share(bob, alice, ted, carol, crypto_provider=cryptoprovider, requires_grad=True)
data = data.fix_precision().share(bob, alice, ted, carol, crypto_provider=cryptoprovider,requires_grad=True)
target = target.fix_precision().share(bob, alice, ted, carol, crypto_provider=cryptoprovider,requires_grad=True)

# send the model back to the local worke
opt = optim.SGD(params=model.parameters(), lr=0.1).fix_precision()
opt.zero_grad()
pred = model(data)
loss = ((pred - target)**2).sum()
loss.backward()
opt.step()
# model.get()
print(loss.get().float_precision())

    
          
    
          

W0711 22:36:15.035753 139805584787328 hook.py:98] Torch was already hooked... skipping hooking process
W0711 22:36:15.037424 139805584787328 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0711 22:36:15.038786 139805584787328 base.py:628] Worker ted already exists. Replacing old worker which could cause                     unexpected behavior
W0711 22:36:15.040513 139805584787328 base.py:628] Worker carol already exists. Replacing old worker which could cause                     unexpected behavior
W0711 22:36:15.042150 139805584787328 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0711 22:36:15.044150 139805584787328 base.py:628] Worker ted already exists. Replacing old worker which could cause                     unexpected behavior
W0711 22:36:15.045879 139805584787328 base.py:628] Worker carol already exists. Replacing old worker which c

tensor([2.9867e+13])
